In [ ]:
# installing packages
!pip install -q ultralytics
!pip install -q scikit-learn
!pip install -q albumentations
!pip install -q kaggle

In [3]:
# importing libraries
import torch
import numpy 
import pandas as pd
import zipfile
import os
import io
import requests
import json
import matplotlib.pyplot as plt
import torchvision
import cv2
import yaml
import glob
import shutil
from PIL import Image
from albumentations import (
    Compose, RandomRotate90, Flip, Resize,
    RandomScale, RandomBrightnessContrast, 
    RandomGridShuffle, RandomScale, GaussNoise, 
    OneOf, CoarseDropout, RandomFog, RandomRain, 
    RandomShadow, RandomSnow, RandomSunFlare, Normalize, BboxParams
)
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

# **Part 1: Merging datasets in a Folder and creating categories**

In [13]:
# preparing classes
yolo_categories = {
    "0": "Yoke",
    "1": "Yoke Suspension",
    "2": "Spacer",
    "3": "Damper - Stockbridge",
    "4": "Lightning Rod Shackle",
    "5": "Lightning Rod Suspension",
    "6": "Polymer Insulator",
    "7": "Glass Insulator",
    "8": "Tower ID Plate",
    "9": "Vari-grip",
    "10": "Polymer Insulator Lower Shackle",
    "11": "Polymer Insulator Upper Shackle",
    "12": "Polymer Insulator Tower Shackle",
    "13": "Glass Insulator Big Shankle",
    "14": "Glass Insulator Small Shankle",
    "15": "Glass Insulator Tower Shankle",
    "16": "Damper - Spiral",
    "17": "Sphere",
    "18": "Cable",
    "19": "Fuse Cutout - Closed",
    "20": "Broken Cable",
    "21": "Tower",
    "22": "Broken Polymer-insulator",
    "23": "Vegetation",
    "24": "Transformer",
    "25": "Connector",
    "26": "Overhead Switch",
    "27": "Broken Vari-Grip",
    "28": "Broken Polymer Insulator Upper Shackle",
    "29": "Broken Glass Insulator",
    "30": "Broken Lightning Rod suspension",
    "31": "Broken Yoke Suspension",
    "32": "Bird Nest",
    "33": "ALS",
    "34": "Damaged ALS",
    "35": "Fuse Cutout - Open",
    "36": "Surge Arrester",
    "37": "Broken-Damper-Stockbridge"
}

# **Data Augmentation of Images and Creating Train, Validation, Test Dataset for Yolo training.**

In [5]:
images_dataset = os.listdir("/kaggle/input/power-line-assets-version-2/Power Line Assets Images")
labels_dataset = os.listdir("/kaggle/input/power-line-assets-version-2/Power Line Assets Labels")

In [6]:
print(len(images_dataset), len(labels_dataset))

21652 21652


In [15]:
import gc

def read_bboxes_from_txt(txt_path, img_width, img_height):
    with open(txt_path, 'r') as file:
        lines = file.readlines()
    
    bboxes = []
    class_labels = []
    format_is_yolo = True
    
    for line in lines:
        # Check if the coordinates are in YOLO format by checking if they are in the range [0, 1]
        coords = list(map(float, line.strip().split()[1:]))
        if any(coord > 1.0 for coord in coords):
            format_is_yolo = False
            break
    
    if format_is_yolo:
        for line in lines:
            cls, cx, cy, w, h = map(float, line.strip().split())
            bboxes.append([cx, cy, w, h])
            class_labels.append(cls)
    else:
        for line in lines:
            cls, x_min, y_min, x_max, y_max = map(float, line.strip().split())
            # Convert to YOLO format
            cx = (x_min + x_max) / 2.0
            cy = (y_min + y_max) / 2.0
            w = x_max - x_min
            h = y_max - y_min
            # Normalize coordinates
            cx /= img_width
            cy /= img_height
            w /= img_width
            h /= img_height
            bboxes.append([cx, cy, w, h])
            class_labels.append(cls)
    
    return bboxes, class_labels

def save_bboxes_to_txt(bboxes, class_labels, txt_path):
    with open(txt_path, 'w') as file:
        for bbox, cls in zip(bboxes, class_labels):
            x, y, w, h = bbox
            file.write(f"{str(int(cls))} {x} {y} {w} {h}\n")
                  
transform2 = Compose([
    Resize(width=640, height=640), 
    RandomRotate90(p=0.5),
    Flip(p=0.5),
    RandomBrightnessContrast(p=0.5, brightness_limit=0.1, contrast_limit=0.1),
    RandomScale(p=0.5, scale_limit=(0.8, 1.2)),
    GaussNoise(p=0.5, var_limit=(10.0, 50.0)),
    OneOf([
        RandomRain(p=0.5, brightness_coefficient=0.9),
        RandomFog(p=0.5, fog_coef_lower=0.01, fog_coef_upper=0.1, alpha_coef=0.1),
        RandomSnow(p=0.5, snow_point_lower=0.1, snow_point_upper=0.3, brightness_coeff=1.2),
        RandomShadow(p=0.5, shadow_dimension=3, num_shadows_lower=1, num_shadows_upper=2, shadow_roi=(0, 0.5, 1, 1))
    ], p=0.5),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
], bbox_params=BboxParams(format="yolo", min_visibility=0.1, label_fields=['class_labels']))

def apply_aug2(image, bboxes, class_labels, tran):
    augmented = tran(image=image, bboxes=bboxes, class_labels=class_labels)
    return augmented['image'], augmented['bboxes'], augmented['class_labels']



def process_image(img_names, image_dir, labels_dir, outputimg, outputtxt, tran, target_list, augment_times):
    for img_name in img_names:
        name = img_name.split(".jpg")[0]
        img_path = os.path.join(image_dir, img_name)
        bboxes_path = os.path.join(labels_dir, name + ".txt")
        
        if os.path.exists(bboxes_path):
            image = cv2.imread(img_path)
            img_height, img_width = image.shape[:2]
            bboxes, class_labels = read_bboxes_from_txt(bboxes_path, img_width, img_height)

            try:
                augment_count = augment_times if any(cls in target_list for cls in class_labels) else 1
                for at in range(augment_count):
                    augmented_image, augmented_bboxes, augmented_class_labels = apply_aug2(image, bboxes, class_labels, tran)
                    if augmented_bboxes:
                        output_aug_img_path = os.path.join(outputimg, f"aug_{at}-{name}.jpg")
                        output_aug_txt_path = os.path.join(outputtxt, f"aug_{at}-{name}.txt")
                        cv2.imwrite(output_aug_img_path, augmented_image)
                        save_bboxes_to_txt(augmented_bboxes, augmented_class_labels, output_aug_txt_path)
            except Exception as e:
                print(bboxes_path, img_name, class_labels)

def process_images_in_batches(img_names, image_dir, labels_dir, outputimg, outputtxt, tran, target_list, augment_times, batch_size=16):
    os.makedirs(outputimg, exist_ok=True)
    os.makedirs(outputtxt, exist_ok=True)

    for i in range(0, len(img_names), batch_size):
        batch = img_names[i:i + batch_size]
        process_image(batch, image_dir, labels_dir, outputimg, outputtxt, tran, target_list, augment_times)
            

target_list = [2, 4, 12, 11, 10, 17, 18, 20, 23, 21, 37, 35]
augmented_times = 4
            
            
# image_dir = "/kaggle/working/power_assets/images/train"
# labels_dir = "/kaggle/working/power_assets/labels/train"
image_dir = "/kaggle/input/power-line-assets-editedv2-trainaug/images/valid"
labels_dir = "/kaggle/input/power-line-assets-editedv2-trainaug/labels/valid"
output_dir_img = "/kaggle/working/Augmented_images_val"
output_dir_txt = "/kaggle/working/Augmented_texts_val"

img_names = os.listdir(image_dir)
process_images_in_batches(img_names, image_dir, labels_dir, output_dir_img, output_dir_txt, transform2, target_list, augmented_times, batch_size=1024)


In [7]:
# Sorted Power Assets
# preparing dataset for yolo
os.mkdir("/kaggle/working/power_assets")
os.mkdir("/kaggle/working/power_assets/images")
os.mkdir("/kaggle/working/power_assets/labels")

In [8]:
import gc

def read_bboxes_from_txt(txt_path, img_width, img_height):
    with open(txt_path, 'r') as file:
        lines = file.readlines()
    
    bboxes = []
    class_labels = []
    format_is_yolo = True
    
    for line in lines:
        # Check if the coordinates are in YOLO format by checking if they are in the range [0, 1]
        coords = list(map(float, line.strip().split()[1:]))
        if any(coord > 1.0 for coord in coords):
            format_is_yolo = False
            break
    
    if format_is_yolo:
        for line in lines:
            cls, cx, cy, w, h = map(float, line.strip().split())
            bboxes.append([cx, cy, w, h])
            class_labels.append(cls)
    else:
        for line in lines:
            cls, x_min, y_min, x_max, y_max = map(float, line.strip().split())
            # Convert to YOLO format
            cx = (x_min + x_max) / 2.0
            cy = (y_min + y_max) / 2.0
            w = x_max - x_min
            h = y_max - y_min
            # Normalize coordinates
            cx /= img_width
            cy /= img_height
            w /= img_width
            h /= img_height
            bboxes.append([cx, cy, w, h])
            class_labels.append(cls)
    
    return bboxes, class_labels

def save_bboxes_to_txt(bboxes, class_labels, txt_path):
    with open(txt_path, 'w') as file:
        for bbox, cls in zip(bboxes, class_labels):
            x, y, w, h = bbox
            file.write(f"{str(int(cls))} {x} {y} {w} {h}\n")
            
transform1 = Compose([
    Resize(width=640, height=640),
], bbox_params=BboxParams(format="yolo", min_visibility=0.1, label_fields=['class_labels']))
           
def apply_aug2(image, bboxes, class_labels, tran):
    augmented = tran(image=image, bboxes=bboxes, class_labels=class_labels)
    return augmented['image'], augmented['bboxes'], augmented['class_labels']



def process_image(img_names, image_dir, labels_dir, outputimg, outputtxt, tran):
    for img_name in img_names:
        name = img_name.split(".jpg")[0]
        img_path = os.path.join(image_dir, img_name)
        bboxes_path = os.path.join(labels_dir, name + ".txt")
        
        if os.path.exists(bboxes_path):
            image = cv2.imread(img_path)
            img_height, img_width = image.shape[:2]
            bboxes, class_labels = read_bboxes_from_txt(bboxes_path, img_width, img_height)

            try:
                augmented_image, augmented_bboxes, augmented_class_labels = apply_aug2(image, bboxes, class_labels, tran)
                if augmented_bboxes:
                    output_aug_img_path = os.path.join(outputimg, f"aug-{name}.jpg")
                    output_aug_txt_path = os.path.join(outputtxt, f"aug-{name}.txt")
                    cv2.imwrite(output_aug_img_path, augmented_image)
                    save_bboxes_to_txt(augmented_bboxes, augmented_class_labels, output_aug_txt_path)
            except Exception as e:
                print(bboxes_path, img_name, class_labels)

def process_images_in_batches2(img_names, image_dir, labels_dir, outputimg, outputtxt, tran, batch_size=16):
    os.makedirs(outputimg, exist_ok=True)
    os.makedirs(outputtxt, exist_ok=True)

    for i in range(0, len(img_names), batch_size):
        batch = img_names[i:i + batch_size]
        process_image(batch, image_dir, labels_dir, outputimg, outputtxt, tran)
            

In [9]:
image_dir = "/kaggle/input/power-line-assets-version-2/Power Line Assets Images"
labels_dir = "/kaggle/input/power-line-assets-version-2/Power Line Assets Labels"
output_dir_img = "/kaggle/working/power_assets/images"
output_dir_txt = "/kaggle/working/power_assets/labels"
img_names = os.listdir(image_dir)
process_images_in_batches2(img_names, image_dir, labels_dir, output_dir_img, output_dir_txt, transform1, batch_size=1024)

/kaggle/input/power-line-assets-version-2/Power Line Assets Labels/DSC_0933_JPG_jpg.rf.bf77cd6de60a69b5b52b7505685f0fc7.txt DSC_0933_JPG_jpg.rf.bf77cd6de60a69b5b52b7505685f0fc7.jpg [6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]
/kaggle/input/power-line-assets-version-2/Power Line Assets Labels/313-2_DJI_0298_jpg.rf.a3c5a462c316a5297633fb6c2dddffe0.txt 313-2_DJI_0298_jpg.rf.a3c5a462c316a5297633fb6c2dddffe0.jpg [6.0, 6.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 0.0, 1.0, 16.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 16.0, 16.0, 16.0]
/kaggle/input/power-line-assets-version-2/Power Line Assets Labels/8545245_png.rf.f9dd7f199be684b1aff03bb32a32467a.txt 8545245_png.rf.f9dd7f199be684b1aff03bb32a32467a.jpg [23.0, 23.0, 23.0]
/kaggle/input/power-line-assets-version-2/Power Line Assets Labels/240-1_DJI_0123_jpg.rf.c5df14693b370fa7fd404216150a03f1.txt 240-1_DJI_0123_jpg.rf.c5df14693b370fa7fd404216150a03f1.jpg [5.0, 25.0, 3.0]
/kaggle/input/power-line-assets-version-2/Power Line Assets Labels/292-1_DJI_0

In [9]:
print(len(os.listdir("/kaggle/working/power_assets_v2/images/test")))
print(len(os.listdir("/kaggle/working/power_assets_v2/labels/test")))

648
648


In [5]:
# Copy the directories to the dataset directory
dataset_directory = '/kaggle/working/power_assets_v2'
os.makedirs(dataset_directory, exist_ok=True)
images_folder = '/kaggle/input/power-line-assets-editedv2-trainaug/images'
labels_folder = '/kaggle/input/power-line-assets-editedv2-trainaug/labels'
def copytree(src, dst, symlinks=False, ignore=None):
    if not os.path.exists(dst):
        os.makedirs(dst)
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree(images_folder, os.path.join(dataset_directory, 'images'))
copytree(labels_folder, os.path.join(dataset_directory, 'labels'))

In [13]:
l1 = os.listdir("/kaggle/working/power_assets/images/train")
l2 = os.listdir("/kaggle/working/power_assets/labels/train")
l3 = os.listdir("/kaggle/working/Augmented_images")
l4 = os.listdir("/kaggle/working/Augmented_texts")
print(len(l1), len(l2), len(l3), len(l4))

17260 17260 31804 31804


In [6]:
images = os.listdir("/kaggle/working/Augmented_images_val")
labels = os.listdir("/kaggle/working/Augmented_texts_val")
images.sort()
labels.sort()
print(images[:2], labels[:2])
def sort_and_copy_files(source_dir, target_dir, sorted_files):

    for file_name in sorted_files:
        source_file_path = os.path.join(source_dir, file_name)
        target_file_path = os.path.join(target_dir, file_name)
    
        if not os.path.exists(target_file_path):
            shutil.copy2(source_file_path, target_file_path)
        else:
            print(f"File {file_name} already exists in {target_dir}, skipping copy.")


sort_and_copy_files("/kaggle/working/Augmented_images_val", "/kaggle/working/power_assets_v2/images/valid", images)
sort_and_copy_files("/kaggle/working/Augmented_texts_val", "/kaggle/working/power_assets_v2/labels/valid", labels)


['aug_0-aug-000000986h_jpeg.rf.926232e83d6364f82adea1bc9a52c832.jpg', 'aug_0-aug-000006140h_JPG.rf.fa4d6325f6eb9b738aa203460de960d5.jpg'] ['aug_0-aug-000000986h_jpeg.rf.926232e83d6364f82adea1bc9a52c832.txt', 'aug_0-aug-000006140h_JPG.rf.fa4d6325f6eb9b738aa203460de960d5.txt']


In [15]:
train_label = os.listdir("/kaggle/working/power_assets/labels/train")
train_img = os.listdir("/kaggle/working/power_assets/images/train")
print(len(train_label), len(train_img))

49064 49064


In [10]:
# creating train test valid
import os
import shutil
import random

def split_data(image_dir, label_dir, train_ratio=0.7, valid_ratio=0.2, test_ratio=0.1):
    os.makedirs(os.path.join(image_dir, "train"), exist_ok=True)
    os.makedirs(os.path.join(image_dir, "valid"), exist_ok=True)
    os.makedirs(os.path.join(image_dir, "test"), exist_ok=True)
    os.makedirs(os.path.join(label_dir, "train"), exist_ok=True)
    os.makedirs(os.path.join(label_dir, "valid"), exist_ok=True)
    os.makedirs(os.path.join(label_dir, "test"), exist_ok=True)

    # Get list of all images
    images = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]
    
    # Shuffle images to ensure random split
    random.seed(40)
    random.shuffle(images)

    # Compute split indices
    total_images = len(images)
    train_end = int(train_ratio * total_images)
    valid_end = int((train_ratio + valid_ratio) * total_images)

    # Split images into train, valid, and test sets
    train_images = images[:train_end]
    valid_images = images[train_end:valid_end]
    test_images = images[valid_end:]

    # Function to move files from one folder to another
    def move_files(file_list, src_image_dir, src_label_dir, dst_image_dir, dst_label_dir):
        for img in file_list:
            # Corresponding label file
            label_file = os.path.splitext(img)[0] + '.txt'
            if os.path.exists(os.path.join(src_label_dir, label_file)):
                # Move image
                shutil.move(os.path.join(src_image_dir, img), os.path.join(dst_image_dir, img))
                # Move label
                shutil.move(os.path.join(src_label_dir, label_file), os.path.join(dst_label_dir, label_file))
            else:
                print(f"Warning: Label file {label_file} does not exist, skipping {img}")

    # Move images and labels to respective folders
    move_files(train_images, image_dir, label_dir, os.path.join(image_dir, 'train'), os.path.join(label_dir, 'train'))
    move_files(valid_images, image_dir, label_dir, os.path.join(image_dir, 'valid'), os.path.join(label_dir, 'valid'))
    move_files(test_images, image_dir, label_dir, os.path.join(image_dir, 'test'), os.path.join(label_dir, 'test'))

    print(f"Data split into train, valid, and test sets:")
    print(f"Train images and labels: {len(train_images)}")
    print(f"Validation images and labels: {len(valid_images)}")
    print(f"Test images and labels: {len(test_images)}")

# Example usage
image_dir = "/kaggle/working/power_assets/images/"
label_dir = "/kaggle/working/power_assets/labels/"

split_data(image_dir, label_dir)

Data split into train, valid, and test sets:
Train images and labels: 15103
Validation images and labels: 4315
Test images and labels: 2158


In [ ]:
# validing the dataset
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def load_labels(label_path):
    labels = []
    with open(label_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            labels.append(line.strip().split())
    return labels

def plot_image_with_labels(image_path, label_path, class_dict):
    
    image = Image.open(image_path)
    image_width, image_height = image.size

    labels = load_labels(label_path)

    fig, ax = plt.subplots(1)
    ax.imshow(image)


    for label in labels:
        class_id = int(label[0])
        x_center = float(label[1]) * image_width
        y_center = float(label[2]) * image_height
        width = float(label[3]) * image_width
        height = float(label[4]) * image_height

        rect = patches.Rectangle(
            (x_center - width / 2, y_center - height / 2),
            width, height,
            linewidth=2, edgecolor='r', facecolor='none'
        )

        ax.add_patch(rect)

        ax.text(x_center - width / 2, y_center - height / 2 - 10, 
                class_dict.get(class_id, 'Unknown'), 
                color='white', backgroundcolor='red', fontsize=12, 
                weight='bold')

    plt.axis('off')  
    plt.show()


image_path = "/kaggle/working/power_assets/images/train/"
label_path = "/kaggle/working/power_assets/labels/train/"
images = os.listdir(image_path)
labels = os.listdir(label_path)

class_dict = yolo_categories 

plot_image_with_labels(image_path+images[75], label_path+labels[75], class_dict)


In [10]:
import os
import json
import shutil

# Upload kaggle.json and move it to the .kaggle directory
kaggle_json = '/kaggle/input/kaggle-is-amazing/kaggle.json'
kaggle_dir = '/root/.kaggle'
os.makedirs(kaggle_dir, exist_ok=True)
shutil.copy(kaggle_json, os.path.join(kaggle_dir, 'kaggle.json'))
os.chmod(os.path.join(kaggle_dir, 'kaggle.json'), 0o600)

# Load the Kaggle credentials from the kaggle.json file
with open(os.path.join(kaggle_dir, 'kaggle.json')) as f:
    kaggle_credentials = json.load(f)
    kaggle_username = kaggle_credentials['username']

# Define the dataset details
dataset_name = 'power-line-assets-editedv2-trainvalaug'
dataset_title = 'Power-Line-Assets-editedv2-trainvalaug'
dataset_directory = '/kaggle/working/power_assets_v2'
# images_folder = '/kaggle/working/power_assets/images/'
# labels_folder = '/kaggle/working/power_assets/labels/'
# Ensure the dataset directory exists
# os.makedirs(dataset_directory, exist_ok=True)

# Move your prepared data to the dataset directory
# For example, if you have already saved your images and labels in respective folders:


# Copy the directories to the dataset directory
# def copytree(src, dst, symlinks=False, ignore=None):
#     if not os.path.exists(dst):
#         os.makedirs(dst)
#     for item in os.listdir(src):
#         s = os.path.join(src, item)
#         d = os.path.join(dst, item)
#         if os.path.isdir(s):
#             copytree(s, d, symlinks, ignore)
#         else:
#             shutil.copy2(s, d)

# copytree(images_folder, os.path.join(dataset_directory, 'images'))
# copytree(labels_folder, os.path.join(dataset_directory, 'labels'))

# Create a dataset-metadata.json file
metadata = {
    'title': dataset_title,
    'id': f'{kaggle_username}/{dataset_name}',
    'licenses': [
        {
            'name': 'CC0-1.0',
            'url': 'http://creativecommons.org/publicdomain/zero/1.0/'
        }
    ]
}

metadata_file = os.path.join(dataset_directory, 'dataset-metadata.json')
with open(metadata_file, 'w') as f:
    json.dump(metadata, f)

# Use the Kaggle API to upload the dataset
!kaggle datasets create -p {dataset_directory} --dir-mode zip

# Optionally, you can add additional flags such as --public to make it public


Starting upload for file images.zip
100%|██████████████████████████████████████| 3.15G/3.15G [00:33<00:00, 99.8MB/s]
Upload successful: images.zip (3GB)
Starting upload for file labels.zip
100%|██████████████████████████████████████| 20.0M/20.0M [00:00<00:00, 22.7MB/s]
Upload successful: labels.zip (20MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/sepehrsimkhah/power-line-assets-editedv2-trainvalaug


In [4]:
import os
import shutil

# Define paths to current and target directories
base_dir = "/kaggle/input/power-line-assets-editedv2"
target_dir = "/kaggle/working/power-line-assets-editedv2"
current_images_dir = os.path.join(base_dir, "Power Line Assets Images")
current_labels_dir = os.path.join(base_dir, "Power Line Assets Labels")
target_images_dir = os.path.join(target_dir, "images")
target_labels_dir = os.path.join(target_dir, "labels")

# Function to copy files from source to destination directory
def copy_files(source_dir, target_dir):
    # Iterate through train, valid, test directories
    for split in ['train', 'valid', 'test']:
        source_split_dir = os.path.join(source_dir, split)
        target_split_dir = os.path.join(target_dir, split)
        os.makedirs(target_split_dir, exist_ok=True)

        # Iterate through files in source split directory
        for filename in os.listdir(source_split_dir):
            source_file_path = os.path.join(source_split_dir, filename)
            target_file_path = os.path.join(target_split_dir, filename)
            
            # Copy file from source to target directory
            shutil.copy(source_file_path, target_file_path)

# Copy images
copy_files(current_images_dir, target_images_dir)

# Copy labels
copy_files(current_labels_dir, target_labels_dir)


In [4]:
# creating yaml file
path = "/kaggle/working/power-line-assets-editedv2/"
cust_yaml = {
    "path":path,
    "train": "images/train",
    "val": "images/valid",
    "test": "images/test",
    "nc": 38,
    "names": ["Yoke","Yoke Suspension","Spacer", "Damper - Stockbridge", "Lightning Rod Shackle",
              "Lightning Rod Suspension", "Polymer Insulator", "Glass Insulator", "Tower ID Plate",
              "Vari-grip", "Polymer Insulator Lower Shackle","Polymer Insulator Upper Shackle",
              "Polymer Insulator Tower Shackle","Glass Insulator Big Shankle","Glass Insulator Small Shankle",
              "Glass Insulator Tower Shankle","Damper - Spiral","Sphere","Cable","Fuse Cutout - Closed",
              "Broken Cable","Tower","Broken Polymer-insulator","Vegetation","Transformer","Connector",
              "Overhead Switch","Broken Vari-Grip","Broken Polymer Insulator Upper Shackle","Broken Glass Insulator",
              "Broken Lightning Rod suspension","Broken Yoke Suspension","Bird Nest","ALS","Damaged ALS",
              "Fuse Cutout - Open","Surge Arrester", "Broken-Damper-Stockbridge"
             ]
}
with open('/kaggle/working/power.yaml', 'w') as file:
    yaml.dump(cust_yaml, file)

In [6]:
import os

data_dir = "/kaggle/working/power-line-assets-editedv2"
print("Train images path exists:", os.path.exists(os.path.join(data_dir, "images/train")))
print("Valid images path exists:", os.path.exists(os.path.join(data_dir, "images/valid")))
print("Test images path exists:", os.path.exists(os.path.join(data_dir, "images/test")))
print("Train labels path exists:", os.path.exists(os.path.join(data_dir, "labels/train")))
print("Valid labels path exists:", os.path.exists(os.path.join(data_dir, "labels/valid")))
print("Test labels path exists:", os.path.exists(os.path.join(data_dir, "labels/test")))


Train images path exists: True
Valid images path exists: True
Test images path exists: True
Train labels path exists: True
Valid labels path exists: True
Test labels path exists: True


In [5]:
import subprocess
def clear_cuda_cache():
    torch.cuda.empty_cache()
    subprocess.run(['nvidia-smi', '--gpu-reset'], capture_output=True, text=True)
    
# Clear the CUDA cache
clear_cuda_cache()

In [9]:
# torch.cuda.empty_cache()
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:64'
# training
model = YOLO("yolov8m.yaml")
model = YOLO("yolov8m.pt")
model = YOLO("yolov8m.yaml").load("yolov8m.pt")
# resume = True if there are checkpoints
result = model.train(data="/kaggle/working/power.yaml", 
    epochs=40,
    imgsz=640,
    batch=-1,
    save_period=2,
    device=1,
    mixup=0.4, 
    mosaic=0.6
)

Transferred 475/475 items from pretrained weights
Ultralytics YOLOv8.2.60 🚀 Python-3.10.13 torch-2.1.2 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/kaggle/working/power.yaml, epochs=40, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=2, cache=False, device=1, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fal

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...

                   all       4891      17412      0.698      0.287      0.316      0.209

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/40      9.99G      1.697      2.644      1.903        125        640: 100%|██████████| 2620/2620 [28:08<00:00,  1.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 117/117 [01:10<00:00,  1.67it/s]


                   all       4891      17412      0.699      0.288      0.318       0.21

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      21/40      9.72G      1.691       2.63      1.899        128        640:  27%|██▋       | 701/2620 [07:31<20:37,  1.55it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



KeyboardInterrupt: 

In [ ]:
# Saving the model
model_save_path = "/kaggle/working/model/power_assets_detection_with_defect_yolov8.pt"
torch.save(model.model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# evaluation model
metrics = model.val(data="/kaggle/working/power.yaml", imgsz=640, batch=16, device=0)

In [ ]:
# testing
test_list = os.listdir("/kaggle/input/power_line_assetsv2/images/test")
model.predict("/kaggle/input/power_line_assetsv2/"+test_list[3], save=True, imgsz=640, conf=0.5)

In [ ]:
# For Loading 
model_save_path = "/kaggle/working/model/power_assets_detection_with_defect

model = YOLO("yolov8m.yaml")

model.model.load_state_dict(torch.load(model_save_path))
model.model.eval()  # Set the model to evaluation mode if you are going to use it for inference


# ***Training with yolo version 9 medium***

In [ ]:
model = YOLO("yolov9m.yaml")
model = YOLO("yolov9m.pt")
model = YOLO("yolov9m.yaml").load("yolov9m.pt")
# resume = True if there are checkpoints
result = model.train(data="/kaggle/working/power.yaml", 
    epochs=40,
    imgsz=640,
    batch=16,
    save_period=2,
    device=0,
    mixup=0.4, 
    mosaic=0.6, 
    dropout=0.2
)

In [ ]:
# Saving the model
model_save_path = "/kaggle/working/model/power_assets_detection_with_defect_yolov9.pt"
torch.save(model.model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# evaluation model
metrics = model.val(data="/kaggle/working/power.yaml", imgsz=640, batch=16, device=[0, 1])

In [ ]:
# testing
model.predict("bus.jpg", save=True, imgsz=640, conf=0.5)

In [ ]:
# For Loading 
model_save_path = "/kaggle/working/model/power_assets_detection_with_defect_yolov9

model = YOLO("yolov9m.yaml")

model.model.load_state_dict(torch.load(model_save_path))
model.model.eval()  # Set the model to evaluation mode if you are going to use it for inference


# ***Training with yolo version 10 medium***

In [ ]:
model = YOLO("yolov10s.yaml")
model = YOLO("yolov10s.pt")
model = YOLO("yolov10s.yaml").load("yolov10s.pt")
# resume = True if there are checkpoints
result = model.train(data="/kaggle/working/power.yaml", 
    epochs=20,
    imgsz=640,
    batch=-1,
    save_period=2,
    device=1,
    mixup=0.4, 
    mosaic=0.6, 
    dropout=0.2,
    optimizer="AdamW",
    workers=2
)

In [ ]:
# Saving the model
model_save_path = "/kaggle/working/model/power_assets_detection_with_defect_yolov10.pt"
torch.save(model.model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# evaluation model
metrics = model.val(data="/kaggle/working/power.yaml", imgsz=640, batch=16, device=[0, 1])

In [ ]:
# testing
model.predict("bus.jpg", save=True, imgsz=640, conf=0.5)

In [ ]:
# For Loading 
model_save_path = "/kaggle/working/model/power_assets_detection_with_defect_yolov10

model = YOLO("yolov10m.yaml")

model.model.load_state_dict(torch.load(model_save_path))
model.model.eval()  # Set the model to evaluation mode if you are going to use it for inference


In [5]:
model = YOLO("/kaggle/input/meowmeow/last(2)-yolo9.pt")

result = model.train(data="/kaggle/working/power.yaml", 
    epochs=100,
    imgsz=640,
    batch=128,
    save_period=2,
    mixup=0.4, 
    mosaic=0.6, 
    dropout=0.2,
    optimizer="auto",
    name="Power Assets with Yolov9",
)

Ultralytics YOLOv8.2.62 🚀 Python-3.10.14 torch-2.3.0+cu121 CPU (Intel Xeon 2.00GHz)
engine/trainer: task=detect, mode=train, model=/kaggle/input/meowmeow/last(2)-yolo9.pt, data=/kaggle/working/power.yaml, epochs=100, time=None, patience=100, batch=128, imgsz=640, save=True, save_period=2, cache=False, device=None, workers=8, project=None, name=Power Assets with Yolov94, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop

E0000 00:00:1721666551.262794     699 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0722 16:42:31.319674310     699 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0722 16:42:31.319695429     699 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0722 16:42:31.319698804     699 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0722 16:42:31.319701207     699 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0722 16:42:31.319703595     699 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     31104  ultralytics.nn.modules.block.ELAN1           [64, 64, 64, 32]              
  3                  -1  1     73984  ultralytics.nn.modules.block.AConv           [64, 128]                     
  4                  -1  1    258432  ultralytics.nn.modules.block.RepNCSPELAN4    [128, 128, 128, 64, 3]        
  5                  -1  1    221568  ultralytics.nn.modules.block.AConv           [128, 192]                    
  6                  -1  1    579648  ultralytics.nn.modules.block.RepNCSPELAN4    [192, 192, 192, 96, 3]        
  7                  -1  1    442880  ultralytics.nn.modules.block.AConv           [192

train: Scanning /kaggle/working/power-line-assets-editedv2/labels/train.cache... 55020 images, 0 backgrounds, 0 corrupt: 100%|██████████| 55020/55020 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/working/power-line-assets-editedv2/labels/valid.cache... 4891 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4891/4891 [00:00<?, ?it/s]


Plotting labels to runs/detect/Power Assets with Yolov94/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 221 weight(decay=0.0), 228 weight(decay=0.001), 227 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/Power Assets with Yolov94
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      2.119      3.561      2.264        845        640:   1%|▏         | 6/430 [09:34<11:16:15, 95.70s/it]


KeyboardInterrupt: 